In [2]:
import awswrangler as wr
import boto3

In [3]:
aws_session = boto3.Session(region_name="us-east-1")

## Glue Catalog Databases

In [12]:
databases = wr.catalog.databases()
print(databases)

   Database      Description
0   default                 
1  sampledb  Sample database


## Create Database in Glue Catalog

In [13]:
wr.catalog.create_database("awswrangler_test_db")

In [14]:
databases = wr.catalog.databases()
print(databases)

              Database      Description
0  awswrangler_test_db                 
1              default                 
2             sampledb  Sample database


## Check table exists or not

In [4]:
glue_table_exists = wr.catalog.does_table_exist('sampledb', 'customer', boto3_session=aws_session)
glue_table_exists

True

In [5]:
glue_table_exists = wr.catalog.does_table_exist('sampledb', 'customer1', boto3_session=aws_session)
glue_table_exists

False

## Get the table location 

In [6]:
glue_table_location = wr.catalog.get_table_location('sampledb', 'customer', boto3_session=aws_session)
glue_table_location

's3://satish-boto3/athena/customer'

## Check is path exists in S3

In [8]:
s3_key = 's3://satish-boto3/athena/customer'
if len(wr.s3.describe_objects(s3_key, boto3_session=aws_session)) > 0:
    s3_key_exists = True
else:
    s3_key_exists = False
    
print(s3_key_exists)

True


In [9]:
s3_key = 's3://satish-boto3/athena/customer12'
if len(wr.s3.describe_objects(s3_key, boto3_session=aws_session)) > 0:
    s3_key_exists = True
else:
    s3_key_exists = False
    
print(s3_key_exists)

False


In [11]:
wr.s3.does_object_exist('s3://satish-boto3/athena/customer',boto3_session=aws_session)

False

## Get tables in Database

In [15]:
wr.catalog.tables(database='sampledb', boto3_session=aws_session)

,Database,Table,Description,Columns,Partitions
0,sampledb,customer,,"first_name, last_name, company_name, address, ...",
1,sampledb,elb_logs,,"request_timestamp, elb_name, request_ip, reque...",


In [16]:
wr.catalog.tables(name_contains="customer")

,Database,Table,Description,Columns,Partitions
0,sampledb,customer,,"first_name, last_name, company_name, address, ...",


In [17]:
wr.catalog.tables(name_prefix="elb")

,Database,Table,Description,Columns,Partitions
0,sampledb,elb_logs,,"request_timestamp, elb_name, request_ip, reque...",


In [19]:
wr.catalog.tables(name_suffix="logs")

,Database,Table,Description,Columns,Partitions
0,sampledb,elb_logs,,"request_timestamp, elb_name, request_ip, reque...",


## Get table details

In [21]:
customers = wr.catalog.table(database="sampledb", table="customer")

In [22]:
type(customers)

pandas.core.frame.DataFrame

In [23]:
customers

,Column Name,Type,Partition,Comment
0,first_name,string,False,
1,last_name,string,False,
2,company_name,string,False,
3,address,string,False,
4,city,string,False,
5,county,string,False,
6,state,string,False,
7,zip,string,False,
8,phone1,string,False,
9,phone2,string,False,


In [25]:
customers.index

RangeIndex(start=0, stop=12, step=1)

In [34]:
for index, row in customers.iterrows():
    print (index,row["Column Name"], row["Type"], row["Partition"])   

0 first_name string False
1 last_name string False
2 company_name string False
3 address string False
4 city string False
5 county string False
6 state string False
7 zip string False
8 phone1 string False
9 phone2 string False
10 email string False
11 web string False


## Cleaning up the Database

In [35]:
for table in wr.catalog.get_tables(database="awswrangler_test_db"):
    wr.catalog.delete_table_if_exists(database="awswrangler_test_db", table=table["Name"])

## Delete Database

In [36]:
wr.catalog.delete_database('awswrangler_test_db')

In [37]:
databases = wr.catalog.databases()
print(databases)

   Database      Description
0   default                 
1  sampledb  Sample database
